<a href="https://githubtocolab.com/jmvazqueznicolas/Pneumonia_detection/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

## Chatbot de preguntas y respuestas





Vamos a aprender cómo construir un sistema de respuesta a preguntas utilizando el DocumentStore, Retriever y Reader de la biblioteca Haystack. Este sistema utilizará información de la serie "Juego de Tronos" y será capaz de responder preguntas como "¿Quién es el padre de Arya Stark?". Sin embargo, puedes utilizarlo para ejecutarlo en cualquier otro conjunto de documentos, como los wikis internos de tu empresa o una colección de informes financieros.

## Instalando Haystack

Para comenzar, instalemos la última versión de Haystack con `pip`:

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

Establecemos el nivel de registro en INFO.

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Inicializando el DocumentStore

Comenzaremos creando nuestro sistema de preguntas y respuestas inicializando un DocumentStore. Un DocumentStore almacena los Documentos que el sistema de preguntas y respuestas utiliza para encontrar respuestas a tus preguntas. En este ejemplo estamos utilizando `InMemoryDocumentStore`, que es el DocumentStore más sencillo para comenzar. No requiere dependencias externas y es una buena opción para proyectos más pequeños y depuración. Sin embargo, no escala tan bien para colecciones de Documentos más grandes, por lo que no es una buena elección para sistemas de producción.

Iniciemos el DocumentStore:


In [ ]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

El DocumentStore está listo ahora. Ahora es el momento de llenarlo con algunos documentos.

## Preparando Documentos

1. Descarga 517 artículos de la Wikipedia de Game of Thrones. Puedes encontrarlos en *data/build_your_first_question_answering_system* como un conjunto de archivos *.txt*.

In [ ]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/build_your_first_question_answering_system"

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip",
    output_dir=doc_dir
)

2. Utiliza `TextIndexingPipeline` para convertir los archivos que acabas de descargar en objetos de documentos de Haystack [Document objects](https://docs.haystack.deepset.ai/docs/documents_answers_labels#document) y escríbelos en el DocumentStore:

In [ ]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

El código en este tutorial utiliza los datos de "Juego de Tronos", pero también puedes suministrar tus propios archivos *.txt* y indexarlos de la misma manera.

Como alternativa, puedes convertir tus datos de texto en [objetos de Documento](https://docs.haystack.deepset.ai/docs/documents_answers_labels#document) y escribirlos en el DocumentStore utilizando `DocumentStore.write_documents()`.

## Inicializando el Retriever

El sistema de búsqueda utilizará un Retriever, por lo que necesitamos inicializarlo. Un Retriever analiza todos los Documentos y devuelve solo aquellos relevantes para la pregunta.

Inicialicemos un BM25Retriever y hagámoslo usar el InMemoryDocumentStore que inicializamos anteriormente.

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

El Retriever está listo, pero aún necesitamos inicializar el Reader.

## Inicializando el Reader

Un Reader escanea los textos que recibe del Retriever y extrae los principales candidatos a respuestas. Los Readers se basan en potentes modelos de aprendizaje profundo, pero son mucho más lentos que los Retrievers al procesar la misma cantidad de texto. En este ejemplo, estamos utilizando un FARMReader con un modelo de preguntas y respuestas.

Inicialicemos el Reader:

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

Hemos inicializado todos los componentes para nuestra tubería. Ahora estamos listos para crear la tubería.

## Creando el Retriever-Reader Pipeline

En este ejemṕlo, estamos utilizando un pipeline predefinido llamado ExtractiveQAPipeline. Conecta el Reader y el Retriever. La combinación de los dos acelera el procesamiento porque el Reader solo procesa los Documentos que el Retriever ha pasado.

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

¡El sistema está listo, ahora puedes seguir adelante y hacer una pregunta!

## Haciendo una pregunta

1. Utiliza el método `run()` del pipeline para hacer una pregunta. El argumento de consulta es donde escribes tu pregunta. Además, puedes establecer el número de documentos que deseas que el Lector (Reader) y el Recuperador (Retriever) devuelvan utilizando el parámetro `top-k`.

In [ ]:
prediction = pipe.run(
    query="Who is the father of Arya Stark?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

Aquí tienes algunas preguntas que podrías probar:

- Who is the father of Arya Stark?
- Who created the Dothraki vocabulary?
- Who is the sister of Sansa?

2. Imprimir las respuestas que devolvió el sistema:

In [ ]:
from pprint import pprint

pprint(prediction)

3. Simplifica las respuestas impresas:

In [ ]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

¡Y ahí lo tienes! ¡Felicitaciones por construir tu primer sistema de respuesta a preguntas basado en aprendizaje automático!